In [1]:
import numpy as np
import pandas as pd

In [ ]:

file_path = r"F:\Hotel Booking\hotel_bookings_sampled_5k.csv" 
df = pd.read_csv(file_path)
print("Initial dataset shape:", df.shape)

Initial dataset shape: (5000, 10)


In [3]:
print("Duplicates before removal:", df.duplicated().sum())

Duplicates before removal: 34


In [4]:
df.drop_duplicates(inplace=True)
print(f"Shape after deduplication: {df.shape}")

Shape after deduplication: (4966, 10)


In [6]:
unique_cols = pd.Series({col: df[col].unique() for col in df.columns})
print(unique_cols)

hotel                                             [City Hotel, Resort Hotel]
is_canceled                                                           [0, 1]
lead_time                  [30, 142, 177, 224, 23, 272, 96, 85, 0, 42, 1,...
arrival_date_year                                         [2016, 2015, 2017]
arrival_date_month         [October, May, July, September, August, March,...
adr                        [144.8, 113.05, 96.0, 114.9, 75.0, 90.1, 188.1...
stays_in_week_nights                                   [4, 3, 1, 2, 0, 5, 6]
stays_in_weekend_nights                                      [0, 1, 2, 3, 4]
country                    [BRA, AUT, ITA, GBR, FRA, DEU, BEL, PRT, NLD, ...
revenue                    [579.2, 452.2, 192.0, 689.4000000000001, 225.0...
dtype: object


In [ ]:
cols_to_keep = [
    'hotel', 
    'is_canceled', 
    'lead_time', 
    'arrival_date_year', 
    'arrival_date_month', 
    'adr', 
    'stays_in_week_nights', 
    'stays_in_weekend_nights', 
    'country'
]
df_structured = df[cols_to_keep].copy()
print("Columns in df_structured:", df_structured.columns.tolist())

Columns in df_structured: ['hotel', 'is_canceled', 'lead_time', 'arrival_date_year', 'arrival_date_month', 'adr', 'stays_in_week_nights', 'stays_in_weekend_nights', 'country']


In [ ]:
# Data Cleaning: Handle missing values
for col in df_structured.columns:
    missing = df_structured[col].isnull().sum()
    if missing > 0:
        if df_structured[col].dtype in [np.int64, np.float64]:
            df_structured[col].fillna(df_structured[col].mean(), inplace=True)
            print(f"Filled {missing} missing values in numeric column '{col}' with mean.")
        else:
            df_structured[col].fillna(df_structured[col].mode()[0], inplace=True)
            print(f"Filled {missing} missing values in categorical column '{col}' with mode.")


In [ ]:
numeric_cols = ['lead_time', 'adr', 'stays_in_week_nights', 'stays_in_weekend_nights']
for col in numeric_cols:
    Q1 = df_structured[col].quantile(0.25)
    Q3 = df_structured[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    before_rows = df_structured.shape[0]
    df_structured = df_structured[(df_structured[col] >= lower_bound) & (df_structured[col] <= upper_bound)]
    after_rows = df_structured.shape[0]
    print(f"Removed {before_rows - after_rows} outlier rows from '{col}'.")

print("Duplicates remaining:", df_structured.duplicated().sum())
print("Missing values per column:\n", df_structured.isnull().sum())

Removed 58 outlier rows from 'lead_time'.
Removed 50 outlier rows from 'adr'.
Removed 0 outlier rows from 'stays_in_week_nights'.
Removed 0 outlier rows from 'stays_in_weekend_nights'.
Duplicates remaining: 0
Missing values per column:
 hotel                      0
is_canceled                0
lead_time                  0
arrival_date_year          0
arrival_date_month         0
adr                        0
stays_in_week_nights       0
stays_in_weekend_nights    0
country                    0
dtype: int64


In [ ]:
df_structured['revenue'] = df_structured['adr'] * (
    df_structured['stays_in_week_nights'] + df_structured['stays_in_weekend_nights']
)

output_path = "hotel_bookings_cleaned11.csv"
df_structured.to_csv(output_path, index=False)
print(f"Cleaned dataset saved to {output_path}")

print(df_structured.head(10))

Cleaned dataset saved to hotel_bookings_cleaned11.csv
         hotel  is_canceled  lead_time  arrival_date_year arrival_date_month  \
0   City Hotel            0         30               2016            October   
1   City Hotel            0        142               2016                May   
2   City Hotel            0        177               2016               July   
3   City Hotel            0        224               2016          September   
4   City Hotel            0         23               2015            October   
6   City Hotel            0         96               2016               July   
7   City Hotel            0         85               2016             August   
8   City Hotel            0          0               2015          September   
9   City Hotel            0         42               2016              March   
10  City Hotel            0          1               2017               June   

       adr  stays_in_week_nights  stays_in_weekend_nights country

In [ ]:
df_revenue = df_structured.groupby(
    ["arrival_date_year", "arrival_date_month"]
)['revenue'].sum().reset_index().rename(columns={'revenue': 'total_revenue'})


print("Revenue Trends Over Time:")
print(df_revenue)

Revenue Trends Over Time:
    arrival_date_year arrival_date_month  total_revenue
0                2015             August       71000.89
1                2015           December       22602.48
2                2015               July       46013.64
3                2015           November       16886.81
4                2015            October       37430.84
5                2015          September       61028.39
6                2016              April       63468.01
7                2016             August      104892.77
8                2016           December       45991.04
9                2016           February       29562.98
10               2016            January       13214.41
11               2016               July       91493.50
12               2016               June       80070.57
13               2016              March       63701.30
14               2016                May       58682.90
15               2016           November       45328.71
16               2016 

In [26]:
df_revenue.head(10)

,arrival_date_year,arrival_date_month,total_revenue
0,2015,August,71000.89
1,2015,December,22602.48
2,2015,July,46013.64
3,2015,November,16886.81
4,2015,October,37430.84
5,2015,September,61028.39
6,2016,April,63468.01
7,2016,August,104892.77
8,2016,December,45991.04
9,2016,February,29562.98


In [ ]:
df_cancel = df_structured.groupby(
    [ "arrival_date_year", "arrival_date_month"]
).agg(
    total_bookings=('is_canceled', 'count'),
    cancellations=('is_canceled', 'sum')
).reset_index()

df_cancel['cancellation_rate (%)'] = (df_cancel['cancellations'] / df_cancel['total_bookings']) * 100
print("\nCancellation Rate (by Country, Year, Month):")
print(df_cancel)


Cancellation Rate (by Country, Year, Month):
    arrival_date_year arrival_date_month  total_bookings  cancellations  \
0                2015             August             128             29   
1                2015           December             111             17   
2                2015               July             109             31   
3                2015           November              89             10   
4                2015            October             166             21   
5                2015          September             178             44   
6                2016              April             208             60   
7                2016             August             201             69   
8                2016           December             170             45   
9                2016           February             168             32   
10               2016            January              89              8   
11               2016               July             1

In [24]:
df_cancel.head(10)

,arrival_date_year,arrival_date_month,total_bookings,cancellations,cancellation_rate (%)
0,2015,August,128,29,22.656250
1,2015,December,111,17,15.315315
2,2015,July,109,31,28.440367
3,2015,November,89,10,11.235955
4,2015,October,166,21,12.650602
5,2015,September,178,44,24.719101
6,2016,April,208,60,28.846154
7,2016,August,201,69,34.328358
8,2016,December,170,45,26.470588
9,2016,February,168,32,19.047619


In [ ]:
df_geography = df_structured['country'].value_counts().reset_index()
df_geography.columns = ['country', 'booking_count']
df_geography = df_geography.head(10)
print("\nTop 10 Countries by Booking Count:")
print(df_geography)


Top 10 Countries by Booking Count:
  country  booking_count
0     PRT           1547
1     FRA            547
2     GBR            524
3     ESP            405
4     DEU            299
5     ITA            166
6     IRL            165
7     NLD            119
8     BRA            112
9     BEL            111


In [ ]:
df_leadtime = df_structured.groupby(
    ["arrival_date_year", "arrival_date_month"]
)['lead_time'].mean().reset_index().rename(columns={'lead_time': 'avg_lead_time'})

print("\nAverage Booking Lead Time (by Year, Month):")
print(df_leadtime)


Average Booking Lead Time (by Country, Year, Month):
    arrival_date_year arrival_date_month  avg_lead_time
0                2015             August      56.109375
1                2015           December      33.729730
2                2015               July      65.266055
3                2015           November      33.000000
4                2015            October      38.078313
5                2015          September      52.269663
6                2016              April      56.432692
7                2016             August     104.064677
8                2016           December      56.041176
9                2016           February      24.880952
10               2016            January      22.471910
11               2016               July      97.860825
12               2016               June      82.277778
13               2016              March      44.145161
14               2016                May      70.472081
15               2016           November      45.3

In [ ]:
df_adr = df_structured.groupby("hotel")['adr'].describe().reset_index()
print("\nADR Distribution by Hotel Type:")
print(df_adr)


ADR Distribution by Hotel Type:
          hotel   count        mean        std   min   25%      50%       75%  \
0    City Hotel  3108.0  109.481088  37.818456  0.00  85.0  105.795  131.7125   
1  Resort Hotel  1750.0   88.542806  50.687964 -6.38  48.0   75.275  121.6275   

     max  
0  217.0  
1  217.0  


In [ ]:
df_structured['total_stay'] = df_structured['stays_in_week_nights'] + df_structured['stays_in_weekend_nights']

df_avg_stay = df_structured.groupby("hotel")['total_stay'].mean().reset_index().rename(columns={'total_stay': 'avg_stay'})
print("\nAverage Stay Duration by Hotel Type:")
print(df_avg_stay)


Average Stay Duration by Hotel Type:
          hotel  avg_stay
0    City Hotel  3.055985
1  Resort Hotel  3.585143
